In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csc_matrix
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import h5py

from sklearn import model_selection, manifold, decomposition


In [2]:
print("Import data")

X = pd.read_hdf('model_pp1.h5', key='data')
X.reset_index(drop=True, inplace=True)

X2 = pd.read_hdf('model_pp2.h5', key='data')
X2.reset_index(drop=True, inplace=True)

X = X.join(X2)

X.drop('window_id', axis=1, inplace=True)

y = X['Label_<lambda>']
X.drop('Label_<lambda>', axis=1, inplace=True)

labels = np.load("labels.npy", allow_pickle=True)

print(X.columns.values)
print(labels)
print(np.where(labels == 'flow=From-Botne')[0][0])


Import data
['Sport_nunique' 'DstAddr_nunique' 'Dport_nunique' 'Dur_sum' 'Dur_mean'
 'Dur_std' 'Dur_max' 'Dur_median' 'TotBytes_sum' 'TotBytes_mean'
 'TotBytes_std' 'TotBytes_max' 'TotBytes_median' 'SrcBytes_sum'
 'SrcBytes_mean' 'SrcBytes_std' 'SrcBytes_max' 'SrcBytes_median'
 'Sport_RU' 'DstAddr_RU' 'Dport_RU']
['flow=Background' 'flow=To-Backgro' 'flow=From-Backg' 'flow=From-Norma'
 'flow=To-Normal-' 'flow=Normal-V42' 'flow=From-Botne']
6


In [3]:
y_bin6 = y == np.where(labels == 'flow=From-Botne')[0][0]
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y_bin6, test_size=0.33, random_state=123456)

print("y", np.unique(y, return_counts=True))
print("y_train", np.unique(X_train, return_counts=True))
print("y_test", np.unique(y_test, return_counts=True))


print("t-SNE")  # Beware: this is very time-consuming
clf = manifold.TSNE(n_components=2, random_state=123456)
clf.fit(X[['Dport_nunique', 'TotBytes_sum', 'Dur_sum', 'Dur_mean', 'TotBytes_std']])

print(clf.embedding_)

y_plot = np.where(y_bin6 == True)[0]
print(len(y_plot))

y_plot2 = np.random.choice(np.where(y_bin6 == False)[
                           0], size=len(y_plot)*100, replace=False)
print(len(y_plot2))
index = list(y_plot)+list(y_plot2)
print(len(index))

plt.scatter(clf.embedding_[index, 0], clf.embedding_[index, 1], c=y[index])
plt.colorbar()
plt.show()


y (array([0, 1, 2, 3, 4, 6], dtype=int64), array([2207092,   18047,     263,     984,      48,     286], dtype=int64))
y_train (array([ -14.64152587,  -14.62718816,  -14.62477672, ..., 1018.93716717,
       1038.07597368, 1038.42284291]), array([5405,    1,    1, ...,    1,    1,    2], dtype=int64))
y_test (array([False,  True]), array([734736,     82], dtype=int64))
t-SNE


c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [ ]:
print("PCA")
clf = decomposition.PCA(n_components=2, random_state=123456)
clf.fit(X[['Dport_nunique', 'TotBytes_sum', 'Dur_sum',
        'Dur_mean', 'TotBytes_std']].transpose())

print(clf.components_)
print(clf.explained_variance_ratio_)

y_plot = np.where(y_bin6 == True)[0]
print(len(y_plot))

y_plot2 = np.random.choice(np.where(y_bin6 == False)[
                           0], size=len(y_plot)*100, replace=False)
print(len(y_plot2))

index = list(y_plot)+list(y_plot2)
print(len(index))

plt.scatter(clf.components_[0, index], clf.components_[1, index], c=y[index])
plt.colorbar()
plt.show()
